# Análise das Intents

Este jupyter-notebook vai auxiliar na análise de detecção de intenções de seu chatbot.

### Configurando e Imports

In [2]:
from IPython.display import IFrame

import rasa
print("Rasa: {}".format(rasa.__version__))

Rasa: 2.5.0


### Treinando o modelo do Rasa NLU

* Para avaliar o bot o primeiro passo é treiner o seu chatbot. Mas não é necessário treinar a parte de conversão completa (rasa_core) apenas a parte de interpretação de mensagens (rasa_nlu).

* O comando `train-nlu` do Makefile executa o treinamento apenas do `rasa_nlu`

In [3]:
!rasa train nlu --config $CONFIG_PATH --nlu $DATA_PATH

(0lqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqk(B
(0x(B Rasa Open Source reports anonymous usage telemetry to help improve the product (0x(B
(0x(B for all its users.                                                             (0x(B
(0x(B                                                                                (0x(B
(0x(B If you'd like to opt-out, you can use `rasa telemetry disable`.                (0x(B
(0x(B To learn more, check out https://rasa.com/docs/rasa/telemetry/telemetry.       (0x(B
(0mqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqj(B
Training NLU model...
/usr/local/lib/python3.7/site-packages/rasa/utils/train_utils.py:537: UserWarning: model_confidence is set to `softmax`. It is recommended to try using `model_confidence=linear_norm` to make it easier to tune fallback thresholds.
  category=UserWarning,
2021-04-29 12:01:47 INFO     rasa.shared.nlu.training_data.training_dat

O comando acima gerou um modelo apenas de **NLU**, este modelo pode ser achado na seguinte pasta:

`modules/notebooks/intents/models/`

In [5]:
!ls models/

ls: cannot access 'models/': No such file or directory


### Métodos de avaliação do chatbot

* O Rasa fornece vários métodos de avaliação e validação das `intents`, para verificar como utiliza-los, cada método fornece um log, imagem, gráfico ou arquivo com dados relevantes para interpretação do chatbot.

* O comando `rasa test nlu` é a base para a avaliação do chatbot.

* Na célula abaixo a flag `-h` foi utilizada para mostrar as funções e a forma de uso de cada uma delas, mude seus valores e flags para ter as informações desejadas na sua análise.

* Atualmente o Rasa possui 2 modos, `evaluation` e `crossvalidation` que tem seções decicadas a eles neste jupyter notebook.

In [4]:
!rasa test nlu -h

usage: rasa test nlu [-h] [-v] [-vv] [--quiet] [-m MODEL] [-u NLU] [--out OUT]
                     [--successes] [--no-errors] [--histogram HISTOGRAM]
                     [--confmat CONFMAT] [-c CONFIG [CONFIG ...]]
                     [--cross-validation] [-f FOLDS] [-r RUNS]
                     [-p PERCENTAGES [PERCENTAGES ...]]

optional arguments:
  -h, --help            show this help message and exit
  -m MODEL, --model MODEL
                        Path to a trained Rasa model. If a directory is
                        specified, it will use the latest model in this
                        directory. (default: models)
  -u NLU, --nlu NLU     File or folder containing your NLU data. (default:
                        data)
  --out OUT             Output path for any files created during the
                        evaluation. (default: results)
  --successes           If set successful predictions (intent and entities)
                        will be written to a file. (defaul

### Evaluation

* O comando abaixo gera informações relevates para a validação das `intents` são elas:
    * Matriz de confusão
    * Histograma
    * Erros de intents

Essas informações e os demais arquivos podem ser encontrados na pasta `results`.

In [5]:
!ls results

CRFEntityExtractor_errors.json	confmat.png  intent_errors.json
CRFEntityExtractor_report.json	hist.png     intent_report.json


In [6]:
!rm -f resutls/*erros.json
!rasa test nlu --nlu $DATA_PATH --model ./models/

2019-10-23 01:05:05 INFO     absl  - Entry Point [tensor2tensor.envs.tic_tac_toe_env:TicTacToeEnv] registered with id [T2TEnv-TicTacToeEnv-v0]
2019-10-23 01:05:06 INFO     rasa.nlu.test  - Running model for predictions:
100%|████████████████████████████████████████| 588/588 [00:00<00:00, 730.43it/s]
2019-10-23 01:05:07 INFO     rasa.nlu.test  - Intent evaluation results:
2019-10-23 01:05:07 INFO     rasa.nlu.test  - Intent Evaluation: Only considering those 588 examples that have a defined intent out of 588 examples
2019-10-23 01:05:07 INFO     rasa.nlu.test  - Classification report saved to results/intent_report.json.
2019-10-23 01:05:07 INFO     rasa.nlu.test  - Incorrect intent predictions saved to results/intent_errors.json.
2019-10-23 01:05:07 INFO     rasa.nlu.test  - Confusion matrix, without normalization: 
[[ 0  0  0 ...  0  0  0]
 [ 0  6  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ... 14  0  0]
 [ 0  0  0 ...  0 18  0]
 [ 0  0  0 ...  0  0 23]]
2019-10-23 01:05:

#### Matriz de Confusão
* A matriz de confusão mostra a correlação entre as intents.
* A diagonal principal tem forte correlação pois mostra a relação de uma intent **com ela mesma**
* O ideal é que não haja **nenhum valor** diferente de **0 fora da diagonal principal**.

In [7]:
IFrame(src='./results/confmat.png', width=900, height=700)

#### Erros
* O arquivo `erros,json` mostra os erros encontrados. Este arquivo lista os mesmos erros mostrados na **matriz de confusão**, então caso ela fique muito grande, não se preocupe, pois você pode apenas procurar os erros no arquivo gerado.
* Os erros mostrados são textos repetidos nos exemplos de diferrentes `intents`.
* Caso o arquivo não seja gerado significa que não foram encontrados erros.

In [8]:
!ls results/

CRFEntityExtractor_errors.json	confmat.png  intent_errors.json
CRFEntityExtractor_report.json	hist.png     intent_report.json


In [9]:
!cat results/intent_errors.json

[
  {
    "text": "e o c#",
    "intent": "linguagens",
    "intent_prediction": {
      "name": "",
      "confidence": 0.0
    }
  }
]

In [10]:
!cat results/CRFEntityExtractor_errors.json

[
  {
    "text": "qual é o seu deus?",
    "entities": [],
    "predicted_entities": [
      {
        "start": 13,
        "end": 17,
        "value": "deus",
        "entity": "religiao",
        "confidence": 0.474498353384622,
        "extractor": "CRFEntityExtractor"
      }
    ]
  },
  {
    "text": "concelhos do yoda",
    "entities": [
      {
        "start": 13,
        "end": 17,
        "value": "yoda",
        "entity": "starwars"
      }
    ],
    "predicted_entities": []
  },
  {
    "text": "guerra nas estrelas",
    "entities": [
      {
        "start": 0,
        "end": 19,
        "value": "guerra nas estrelas",
        "entity": "starwars"
      }
    ],
    "predicted_entities": []
  },
  {
    "text": "como você é?",
    "entities": [
      {
        "start": 0,
        "end": 11,
        "value": "como você é",
        "entity": "how"
      }
    ],
    "predicted_entities": []
  },
  {
    "text": "me conta uma piada",
    "entities": [],
    "predicted_enti

#### Histograma

* O histograma contém a distribuição da predições das `intents`

In [1]:
IFrame(src='./results/hist.png', width=900, height=700)

NameError: name 'IFrame' is not defined

## Referências:

O Rasa está em constante evolução, alguns links úteis para a construção deste jupyter-notebook e para a análise das `intents` são:

* [Evaluation](https://rasa.com/docs/nlu/evaluation/)